In [2]:
#################################################################################
#
#   Anaylsis 2. ROI Search plus Parallelism to speed things up
#
#   This starts with the code from Analysis 1 and adds 'Parallel' and 'delayed' from
#   the joblib Python library.
#
#   Goal: Instead of processing each mask one at a time in
#   a for-loop, attempt to run as parallel threads using multiple cores in
#   parallel.
#
##################################################################################
from amvpa import *
import numpy as np
from sklearn import svm
import sys, os
from joblib import Parallel, delayed
import nibabel as nb

### Question 1: 

In [142]:
#sub = sys.argv[1]
sub = '36'

# +++++++++++++++++++  SETUP same as in Analysis 1  ++++++++++++++++++++

dpath = "../../preproc/sub-rid0000{}/".format(sub)
opath = "../sub-rid0000{}/".format(sub)

try:
    os.mkdir(opath)
    os.symlink("{}T1w_USAQ.nii.gz".format(dpath), "{}T1w_USAQ.nii.gz".format(opath)) 
except:
    pass # you can only do that once without causing trouble

tasks = ["beh","tax"]
data_fn = dpath+"Qtstats_{}_run-{}.nii.gz"
mask_fn = dpath+"glasser_masks.nii.gz"
animals = ['bird','insect','primate','reptile','ungulate']
behaviors = ['eating','fighting','running','swimming']
twenty_conds = ['{}_{}'.format(a,b) for a in animals for b in behaviors]

# END SETUP 

In [143]:
glassr = nb.load(mask_fn).get_fdata()

In [144]:
# as in analysis 1 collect data in dict
results = {}

# Use this many processing cores
nproc = 30

# instead of using the ranges for mask ids, instead use the mask data to
# determine mask ids
# OLD: masks = np.hstack((np.arange(1,181), np.arange(1001,1170)))
# new:

# load glasser masks and get the 3-dimensional volume array
glassr = nb.load(mask_fn).get_fdata()

# Flatten the masks array to a vector and call np.unique to get each unique voxel
# value exactly once. save these values as a list of ints (i.e., asarray w
# dtype='int').  
masks = np.asarray(np.unique(glassr.flatten()), dtype='int')

# This gives us what we need except it has an uneeded zero
# which should be in the first position..
masks = masks[1:] # chop off first element.

# to check if everything works so far withour going further uncomment:
#print(masks); sys.exit()

In [145]:
# write the core of the former for-loop as a function
# This function tasks two arguments: a dataset, ds, and an int, m. where m is
# the id of the mask
def compute_roi_searchlight(mask_val):
    try:
        ds = None
        for task in tasks:
            for r in range(1,6):
                if ds is None:
                    ds = Dataset(data_fn.format(task, r), mask=mask_fn,
                            mask_val=mask_val)
                else:
                    ds.append(Dataset(data_fn.format(task,r), mask=mask_fn,
                        mask_val=mask_val))

        ds.set_sa('targets', np.tile(twenty_conds,10))
        ds.set_sa('chunks', np.repeat(range(10),20))
        ds.zscore_by_chunk() #at first make the chunks the runs, then z-score, and then change the chunks to the behavior
        #z-score by chunk will z-score within each behavior if behaviors are chunks

        results[mask_val] = np.mean(cross_validated_classification(ds, svm.LinearSVC))
    except:
        print('Mask val {} fails'.format(mask_val))
        pass

In [146]:
# Now loop in parallel over the set mask ids
Parallel(n_jobs=nproc, require='sharedmem')(delayed(compute_roi_searchlight)(m) for m in
        masks)

print("00000____=_+_==--=====+++++...,,,,234723hwefdubc897g4efwev...")


res_ds = Dataset(mask_fn)
res_samp = np.zeros(res_ds.shape)
for m in results:
    res_samp[res_ds.samples==m] = results[m]

res_ds.samples = res_samp
ni = res_ds.map_to_nifti()
ni.to_filename(opath+"reslt_2_ROISearch+PP.nii.gz")

print("All done. Go to {} and use Afni, MRICroGl, or whatever to view your "\
        "results".format(opath))

  

/dartfs-hpc/rc/home/9/f004p59/.local/lib/python3.7/site-packages/sklearn/svm/_base.py:977: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/dartfs-hpc/rc/home/9/f004p59/.local/lib/python3.7/site-packages/sklearn/svm/_base.py:977: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/dartfs-hpc/rc/home/9/f004p59/.local/lib/python3.7/site-packages/sklearn/svm/_base.py:977: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/dartfs-hpc/rc/home/9/f004p59/.local/lib/python3.7/site-packages/sklearn/svm/_base.py:977: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/dartfs-hpc/rc/home/9/f004p59/.local/lib/python3.7/site-packages/sklearn/svm/_base.py:977: C

/dartfs-hpc/rc/home/9/f004p59/.local/lib/python3.7/site-packages/sklearn/svm/_base.py:977: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/dartfs-hpc/rc/home/9/f004p59/.local/lib/python3.7/site-packages/sklearn/svm/_base.py:977: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/dartfs-hpc/rc/home/9/f004p59/.local/lib/python3.7/site-packages/sklearn/svm/_base.py:977: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/dartfs-hpc/rc/home/9/f004p59/.local/lib/python3.7/site-packages/sklearn/svm/_base.py:977: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/dartfs-hpc/rc/home/9/f004p59/.local/lib/python3.7/site-packages/sklearn/svm/_base.py:977: C

/dartfs-hpc/rc/home/9/f004p59/.local/lib/python3.7/site-packages/sklearn/svm/_base.py:977: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/dartfs-hpc/rc/home/9/f004p59/.local/lib/python3.7/site-packages/sklearn/svm/_base.py:977: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/dartfs-hpc/rc/home/9/f004p59/.local/lib/python3.7/site-packages/sklearn/svm/_base.py:977: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/dartfs-hpc/rc/home/9/f004p59/.local/lib/python3.7/site-packages/sklearn/svm/_base.py:977: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/dartfs-hpc/rc/home/9/f004p59/.local/lib/python3.7/site-packages/sklearn/svm/_base.py:977: C

/dartfs-hpc/rc/home/9/f004p59/.local/lib/python3.7/site-packages/sklearn/svm/_base.py:977: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/dartfs-hpc/rc/home/9/f004p59/.local/lib/python3.7/site-packages/sklearn/svm/_base.py:977: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/dartfs-hpc/rc/home/9/f004p59/.local/lib/python3.7/site-packages/sklearn/svm/_base.py:977: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/dartfs-hpc/rc/home/9/f004p59/.local/lib/python3.7/site-packages/sklearn/svm/_base.py:977: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/dartfs-hpc/rc/home/9/f004p59/.local/lib/python3.7/site-packages/sklearn/svm/_base.py:977: C

/dartfs-hpc/rc/home/9/f004p59/.local/lib/python3.7/site-packages/sklearn/svm/_base.py:977: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/dartfs-hpc/rc/home/9/f004p59/.local/lib/python3.7/site-packages/sklearn/svm/_base.py:977: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/dartfs-hpc/rc/home/9/f004p59/.local/lib/python3.7/site-packages/sklearn/svm/_base.py:977: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/dartfs-hpc/rc/home/9/f004p59/.local/lib/python3.7/site-packages/sklearn/svm/_base.py:977: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/dartfs-hpc/rc/home/9/f004p59/.local/lib/python3.7/site-packages/sklearn/svm/_base.py:977: C

...

/dartfs-hpc/rc/home/9/f004p59/.local/lib/python3.7/site-packages/sklearn/svm/_base.py:977: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/dartfs-hpc/rc/home/9/f004p59/.local/lib/python3.7/site-packages/sklearn/svm/_base.py:977: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/dartfs-hpc/rc/home/9/f004p59/.local/lib/python3.7/site-packages/sklearn/svm/_base.py:977: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/dartfs-hpc/rc/home/9/f004p59/.local/lib/python3.7/site-packages/sklearn/svm/_base.py:977: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/dartfs-hpc/rc/home/9/f004p59/.local/lib/python3.7/site-packages/sklearn/svm/_base.py:977: C

/dartfs-hpc/rc/home/9/f004p59/.local/lib/python3.7/site-packages/sklearn/svm/_base.py:977: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


00000____=_+_==--=====+++++...,,,,234723hwefdubc897g4efwev...


/dartfs-hpc/rc/home/9/f004p59/.local/lib/python3.7/site-packages/sklearn/svm/_base.py:977: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


PermissionError: [Errno 13] Permission denied: '../sub-rid000036/reslt_2_ROISearch+PP.nii.gz'

### Question 2: 
- Classify taxa (1 out of 5), training on videos with 3 behaviors and testing on videos with the left-out behavior Average accuracies over the 4 data folds.

In [136]:
sub = sys.argv[1]

sub = '41'

# +++++++++++++++++++  SETUP same as in Analysis 1  ++++++++++++++++++++

dpath = "../../preproc/sub-rid0000{}/".format(sub)
opath = "../sub-rid0000{}/".format(sub)

try:
    os.mkdir(opath)
    os.symlink("{}T1w_USAQ.nii.gz".format(dpath), "{}T1w_USAQ.nii.gz".format(opath))
except:
    pass # you can only do that once without causing trouble

tasks = ["beh","tax"]
data_fn = dpath+"Qtstats_{}_run-{}.nii.gz"
mask_fn = dpath+"glasser_masks.nii.gz"
animals = ['bird','insect','primate','reptile','ungulate']
behaviors = ['eating','fighting','running','swimming']


In [138]:
# as in analysis 1 collect data in dict
results = {}

# Use this many processing cores
nproc = 30

# instead of using the ranges for mask ids, instead use the mask data to
# determine mask ids
# OLD: masks = np.hstack((np.arange(1,181), np.arange(1001,1170)))
# new:

# load glasser masks and get the 3-dimensional volume array
glassr = nb.load(mask_fn).get_fdata()

# Flatten the masks array to a vector and call np.unique to get each unique voxel
# value exactly once. save these values as a list of ints (i.e., asarray w
# dtype='int').
masks = np.asarray(np.unique(glassr.flatten()), dtype='int')

In [139]:

# This gives us what we need except it has an uneeded zero
# which should be in the first position..
masks = masks[1:] # chop off first element.

# to check if everything works so far withour going further uncomment:
#print(masks); sys.exit()


# write the core of the former for-loop as a function
# This function tasks two arguments: a dataset, ds, and an int, m. where m is
# the id of the mask


In [140]:
def compute_roi_searchlight(mask_val):
    try:
        ds = None
        for task in tasks:
            for r in range(1,6):
                if ds is None:
                    ds = Dataset(data_fn.format(task, r), mask=mask_fn,
                            mask_val=mask_val)
                else:
                    ds.append(Dataset(data_fn.format(task,r), mask=mask_fn,
                        mask_val=mask_val))

        # targets should be animals, chunks should be behavior

        
        
        #ds.set_sa('targets', np.tile(animals,5))
        #ds.set_sa('chunks', np.repeat(range(10),20)) #  10 bc 5 runs x 2 tasks
        #ds.set_sa('targets',np.tile(np.repeat(animals,4),10))
        #ds.set_sa('chunks',np.tile(behaviors,50))
        ds.set_sa('chunks', np.repeat(range(10),20)) #  10 bc 5 runs x 2 tasks
        #ds.set_sa('chunks',np.tile(behaviors,50))
        ds.zscore_by_chunk()
        ds.set_sa('targets',np.tile(np.repeat(animals, 4),10))
        ds.set_sa('chunks',np.tile(behaviors,50))
        
        # andy wisdom:
        # at first make the chunks the runs, then z-score, and then change the chunks to the behavior
        # z-score by chunk will z-score within each behavior if behaviors are chunks

        # change chunks to behavior here
        results[mask_val] = np.mean(cross_validated_classification(ds, svm.LinearSVC))
    except:
        pass

In [141]:
# Now loop in parallel over the set mask ids
Parallel(n_jobs=nproc, require='sharedmem')(delayed(compute_roi_searchlight)(m) for m in
        masks)

print("00000____=_+_==--=====+++++...,,,,234723hwefdubc897g4efwev...")


res_ds = Dataset(mask_fn)
res_samp = np.zeros(res_ds.shape)
for m in results:
    res_samp[res_ds.samples==m] = results[m]

res_ds.samples = res_samp
ni = res_ds.map_to_nifti()
ni.to_filename(opath+"q2_reslt_2_ROISearch+PP.nii.gz")

print("All done. Go to {} and use Afni, MRICroGl, or whatever to view your "\
        "results".format(opath))


/dartfs-hpc/rc/home/9/f004p59/.local/lib/python3.7/site-packages/sklearn/svm/_base.py:977: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/dartfs-hpc/rc/home/9/f004p59/.local/lib/python3.7/site-packages/sklearn/svm/_base.py:977: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/dartfs-hpc/rc/home/9/f004p59/.local/lib/python3.7/site-packages/sklearn/svm/_base.py:977: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/dartfs-hpc/rc/home/9/f004p59/.local/lib/python3.7/site-packages/sklearn/svm/_base.py:977: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/dartfs-hpc/rc/home/9/f004p59/.local/lib/python3.7/site-packages/sklearn/svm/_base.py:977: C

/dartfs-hpc/rc/home/9/f004p59/.local/lib/python3.7/site-packages/sklearn/svm/_base.py:977: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/dartfs-hpc/rc/home/9/f004p59/.local/lib/python3.7/site-packages/sklearn/svm/_base.py:977: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/dartfs-hpc/rc/home/9/f004p59/.local/lib/python3.7/site-packages/sklearn/svm/_base.py:977: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/dartfs-hpc/rc/home/9/f004p59/.local/lib/python3.7/site-packages/sklearn/svm/_base.py:977: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/dartfs-hpc/rc/home/9/f004p59/.local/lib/python3.7/site-packages/sklearn/svm/_base.py:977: C

/dartfs-hpc/rc/home/9/f004p59/.local/lib/python3.7/site-packages/sklearn/svm/_base.py:977: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/dartfs-hpc/rc/home/9/f004p59/.local/lib/python3.7/site-packages/sklearn/svm/_base.py:977: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/dartfs-hpc/rc/home/9/f004p59/.local/lib/python3.7/site-packages/sklearn/svm/_base.py:977: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/dartfs-hpc/rc/home/9/f004p59/.local/lib/python3.7/site-packages/sklearn/svm/_base.py:977: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/dartfs-hpc/rc/home/9/f004p59/.local/lib/python3.7/site-packages/sklearn/svm/_base.py:977: C

/dartfs-hpc/rc/home/9/f004p59/.local/lib/python3.7/site-packages/sklearn/svm/_base.py:977: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/dartfs-hpc/rc/home/9/f004p59/.local/lib/python3.7/site-packages/sklearn/svm/_base.py:977: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/dartfs-hpc/rc/home/9/f004p59/.local/lib/python3.7/site-packages/sklearn/svm/_base.py:977: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/dartfs-hpc/rc/home/9/f004p59/.local/lib/python3.7/site-packages/sklearn/svm/_base.py:977: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/dartfs-hpc/rc/home/9/f004p59/.local/lib/python3.7/site-packages/sklearn/svm/_base.py:977: C

/dartfs-hpc/rc/home/9/f004p59/.local/lib/python3.7/site-packages/sklearn/svm/_base.py:977: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/dartfs-hpc/rc/home/9/f004p59/.local/lib/python3.7/site-packages/sklearn/svm/_base.py:977: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/dartfs-hpc/rc/home/9/f004p59/.local/lib/python3.7/site-packages/sklearn/svm/_base.py:977: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/dartfs-hpc/rc/home/9/f004p59/.local/lib/python3.7/site-packages/sklearn/svm/_base.py:977: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/dartfs-hpc/rc/home/9/f004p59/.local/lib/python3.7/site-packages/sklearn/svm/_base.py:977: C

...

/dartfs-hpc/rc/home/9/f004p59/.local/lib/python3.7/site-packages/sklearn/svm/_base.py:977: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/dartfs-hpc/rc/home/9/f004p59/.local/lib/python3.7/site-packages/sklearn/svm/_base.py:977: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/dartfs-hpc/rc/home/9/f004p59/.local/lib/python3.7/site-packages/sklearn/svm/_base.py:977: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/dartfs-hpc/rc/home/9/f004p59/.local/lib/python3.7/site-packages/sklearn/svm/_base.py:977: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/dartfs-hpc/rc/home/9/f004p59/.local/lib/python3.7/site-packages/sklearn/svm/_base.py:977: C

/dartfs-hpc/rc/home/9/f004p59/.local/lib/python3.7/site-packages/sklearn/svm/_base.py:977: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/dartfs-hpc/rc/home/9/f004p59/.local/lib/python3.7/site-packages/sklearn/svm/_base.py:977: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/dartfs-hpc/rc/home/9/f004p59/.local/lib/python3.7/site-packages/sklearn/svm/_base.py:977: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/dartfs-hpc/rc/home/9/f004p59/.local/lib/python3.7/site-packages/sklearn/svm/_base.py:977: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/dartfs-hpc/rc/home/9/f004p59/.local/lib/python3.7/site-packages/sklearn/svm/_base.py:977: C

/dartfs-hpc/rc/home/9/f004p59/.local/lib/python3.7/site-packages/sklearn/svm/_base.py:977: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/dartfs-hpc/rc/home/9/f004p59/.local/lib/python3.7/site-packages/sklearn/svm/_base.py:977: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/dartfs-hpc/rc/home/9/f004p59/.local/lib/python3.7/site-packages/sklearn/svm/_base.py:977: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/dartfs-hpc/rc/home/9/f004p59/.local/lib/python3.7/site-packages/sklearn/svm/_base.py:977: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/dartfs-hpc/rc/home/9/f004p59/.local/lib/python3.7/site-packages/sklearn/svm/_base.py:977: C

00000____=_+_==--=====+++++...,,,,234723hwefdubc897g4efwev...
All done. Go to ../sub-rid000041/ and use Afni, MRICroGl, or whatever to view your results


/dartfs-hpc/rc/home/9/f004p59/.local/lib/python3.7/site-packages/sklearn/svm/_base.py:977: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/dartfs-hpc/rc/home/9/f004p59/.local/lib/python3.7/site-packages/sklearn/svm/_base.py:977: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


In [120]:
ni.to_filename(opath+"q2_reslt_2_ROISearch+PP.nii.gz")


In [ ]:
mask_x = expand_mask(mask)
mask_x
pd.Series(l)
event_value_to_plot = pd.Series(l)
event_value_to_plot
evt_plt = roi_to_brain(event_value_to_plot,mask_x)
print(len(evt_plt))
print(evt_plt.shape())

evt_plt.plot()
#used 3.2 because 3.13 is the max for Lemonade
evt_plt.iplot(colorbar=True,surface=False, draw_cross=False,anatomical=None,colorbar_height=0.2, symmetric_cmap=False,colorbar_fontsize=10,cmap='plasma',vmax=3.2)
